In [25]:
import smartdata as smdt

In [26]:
import smartdata
from smartdata.utils.common import print_attributes as pa
from smartdata.utils.dframe import display_full_df as ddff
from smartdata.utils.common import to_namedtuple
from smartdata.utils.file import read_from_json
from smartdata import SmartData

***

In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import sys, os
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')

sys.path.append(r"C:\git\CHRIS_SCRIPTS")

# Test Net Back

In [29]:
from gmt.dna.amp.models.gas.euroview.net_back import *
from gmt.dna.amp.models.tests.gas.euroview.test_netback import *

In [30]:
from datetime import date, timedelta
import sqlalchemy as sa

In [31]:
start_dt = date(2018, 1, 2)
end_dt = date(2020, 11, 26)

In [32]:
dt = work_days_range(start_dt, end_dt)[0]
today = dt.day
if today > 20:
    from_date = dt + pd.DateOffset(months=2)
    to_date = dt + pd.DateOffset(months=2 ,years=3)    
else:
    from_date = dt + pd.DateOffset(months=1)
    to_date = dt + pd.DateOffset(months=1 ,years=3)


In [33]:
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

In [85]:
query_str_1 = f"""
                SELECT
                  d.[CurveName],
                  datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
                  AVG(d.[PointValue]) as PointValue
                FROM [arc_ts_data].[PriceCurveData] d
                WHERE d.EffectiveDate = '{dt.strftime("%Y-%m-%d")}' and [CurveName] in 
                (
                
                            
                            'G_H_NBP.USD', 
                            'G_H_Zeebrugge-Hub.Pnc',
                            'LNG_JKM_M.USD', 
                            'G_H_TRS.EUR', 
                            'LNG_DES_ME_W_INDIA_M.USD', 
                            'LNG_H_DES_Med_MID_M.USD', 
                            'LNG_H_DES_JKM_MID_M.USD', 
                            'LNG_H_DES_NWE_MID_M.USD', 
                            'G_H_TTF.EUR', 
                            'G_H_CEGH.EUR', 
                            'G_H_AOC.EUR', 
                            'G_H_PEG-Nord.EUR', 
                            'G_H_PSV.EUR',
                            'G_NYM_HH_M.USD',
                            'LNG_H_DES_Lat_Am_MID_M.USD', 
                            'FX_curve_cross_EUR.USD', 
                            'FX_curve_input_USD.GBP', 
                            'FREIGHT_H_LNG_M_RolledOver.USD'
                            
                
                
                )
                
                
                
                and d.PointDate >= '{from_date.strftime("%Y-%m-01")}' and d.PointDate < '{to_date.strftime("%Y-%m-01")}'
                group by d.CurveId, d.CurveName, d.EffectiveDate, 12 * year(d.PointDate) + month(d.PointDate)
                order by d.CurveID
                """
#------------------------------------------------------Prices---------------------------------------------------------------
Prices = query_arc_alchemy(query_str_1)  

In [86]:
print(query_str_1)


                SELECT
                  d.[CurveName],
                  datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
                  AVG(d.[PointValue]) as PointValue
                FROM [arc_ts_data].[PriceCurveData] d
                WHERE d.EffectiveDate = '2018-01-02' and [CurveName] in 
                (
                
                            
                            'G_H_NBP.USD', 
                            'G_H_Zeebrugge-Hub.Pnc',
                            'LNG_JKM_M.USD', 
                            'G_H_TRS.EUR', 
                            'LNG_DES_ME_W_INDIA_M.USD', 
                            'LNG_H_DES_Med_MID_M.USD', 
                            'LNG_H_DES_JKM_MID_M.USD', 
                            'LNG_H_DES_NWE_MID_M.USD', 
                            'G_H_TTF.EUR', 
                            'G_H_CEGH.EUR', 
                            'G_H_AOC.EUR', 
                            'G_H_PEG-Nord.EUR', 
        

In [1]:
#the next two lines are there to be able to use datagateway in the jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
from datetime import date, datetime
from gmt.dna.datagateway.data_adapter_factory import DataAdapterFactory

In [88]:
def _get_arc_curve(curve, dt, from_date, to_date, env='prod') -> pd.DataFrame:
    data_adaptor = DataAdapterFactory.create("arc", env=env)
    curve_df = data_adaptor.get_forward_curve(curve, dt).resample('M').mean().loc[from_date: to_date]
    curve_df.index = [date(t.year, t.month, 1) for t in curve_df.index]
    curve_df = curve_df.to_frame()
    curve_df.reset_index(inplace=True)
    curve_df.columns = [NetBackSymbols.SYM_POINTDATE, NetBackSymbols.SYM_POINTVALUE]
    curve_df[NetBackSymbols.SYM_CURVENAME] = curve
    return curve_df

In [89]:
curves = [
                            'G_H_NBP.USD', 
                            'G_H_Zeebrugge-Hub.Pnc',
                            'LNG_JKM_M.USD', 
                            'G_H_TRS.EUR', 
                            'LNG_DES_ME_W_INDIA_M.USD', 
                            'LNG_H_DES_Med_MID_M.USD', 
                            'LNG_H_DES_JKM_MID_M.USD', 
                            'LNG_H_DES_NWE_MID_M.USD', 
                            'G_H_TTF.EUR', 
                            'G_H_CEGH.EUR', 
                            'G_H_AOC.EUR', 
                            'G_H_PEG-Nord.EUR', 
                            'G_H_PSV.EUR',
                            'G_NYM_HH_M.USD',
                            'LNG_H_DES_Lat_Am_MID_M.USD', 
                            'FX_curve_cross_EUR.USD', 
                            'FX_curve_input_USD.GBP', 
                            'FREIGHT_H_LNG_M_RolledOver.USD'
]
results = pd.concat([_get_arc_curve(c, dt, from_date, to_date) for c in curves], axis=0, sort=False)

In [90]:
Prices['PointValue'] = Prices['PointValue'].astype(float)

In [91]:
comp_df = Prices.set_index(['CurveName', 'PointDate']).join(results.set_index(['CurveName', 'PointDate']), lsuffix='price')

In [92]:
comp_df['diff'] = comp_df['PointValueprice'].sub(comp_df['PointValue'], axis=0)

In [95]:
ddff(comp_df)

PointValueprice    PointValue  \
CurveName                      PointDate                                   
LNG_JKM_M.USD                  2018-02-01        11.400000     11.400000   
                               2018-03-01         9.625000      9.625000   
                               2018-04-01         7.628000      7.628000   
                               2018-05-01         7.156000      7.156000   
                               2018-06-01         6.726000      6.726000   
                               2018-07-01         6.828000      6.828000   
                               2018-08-01         6.890000      6.890000   
                               2018-09-01         6.916000      6.916000   
                               2018-10-01         7.237000      7.237000   
                               2018-11-01         7.945000      7.945000   
                               2018-12-01         8.286000      8.286000   
                               2019-01-01         8.324000      8.324000   
                               2019-02-01         8.361000      8.361000   
                               2019-03-01         8.080000      8.080000   
                               2019-04-01         6.871000      6.871000   
                               2019-05-01         6.324000      6.324000   
                               2019-06-01         6.192000      6.192000   
                               2019-07-01         6.254000      6.254000   
                               2019-08-01         6.330000      6.330000   
                               2019-09-01         6.455000      6.455000   
                               2019-10-01         6.591000      6.591000   
                               2019-11-01         7.353000      7.353000   
                               2019-12-01         7.711000      7.711000   
                               2020-01-01         7.924000      7.924000   
                               2020-02-01         7.960000      7.960000   
                               2020-03-01         7.674000      7.674000   
                               2020-04-01         6.763000      6.763000   
                               2020-05-01         6.299000      6.299000   
                               2020-06-01         6.207000      6.207000   
                               2020-07-01         6.181000      6.181000   
                               2020-08-01         6.258000      6.258000   
                               2020-09-01         6.384000      6.384000   
                               2020-10-01         6.480000      6.480000   
                               2020-11-01         7.322000      7.322000   
                               2020-12-01         7.670000      7.670000   
                               2021-01-01         7.538000      7.538000   
G_H_TTF.EUR                    2018-02-01        19.325029     19.325029   
                               2018-03-01        18.950000     18.950000   
                               2018-04-01        18.050000     18.050000   
                               2018-05-01        17.600000     17.600000   
                               2018-06-01        17.400000     17.400000   
                               2018-07-01        17.125000     17.125000   
                               2018-08-01        17.350000     17.350000   
                               2018-09-01        17.625000     17.625000   
                               2018-10-01        18.131000     18.131000   
                               2018-11-01        18.731000     18.731000   
                               2018-12-01        19.131000     19.131000   
                               2019-01-01        19.281000     19.281000   
                               2019-02-01        19.281000     19.281000   
                               2019-03-01        19.006000     19.006000   
                               2019-04-01        17.501800     17.501800   
               

In [96]:
comp_df['PointValueprice'].sub(comp_df['PointValue'], axis=0).sum()

-3.21805915248774e-11